In [92]:
# Pentru obtinerea datelor
import pandas as pd
import requests
import json
import csv
from datetime import date

Mai intai, vom obtine datele necesare de la API-ul Airly.

In [ ]:
# Pentru Targoviste, avem un singur senzor, deci a trebuit doar sa ii gasim id-ul

URL_TGV = "https://airapi.airly.eu/v2/measurements/installation?installationId=118887"

# Pentru camin, am ales senzorul de pe calea plevnei - am incercat interpolare cu cel de pe lacul morii dar se complica

URL_CAMIN = "https://airapi.airly.eu/v2/measurements/installation?installationId=97058"

# Headerele http care includ cheia API

headere = {'Accept': 'application/json', 'apikey': ''}

Urmeaza sa scriem datele obtinute in fisiere, dupa care le vom incarca in program.

In [94]:
# Facem request-urile pentru cele doua locatii

tgv_request = requests.get(URL_TGV, headere)
tgv_json = tgv_request.json()

camin_request = requests.get(URL_CAMIN, headere)
camin_json = camin_request.json()

# Scrierea in fisier

with open("targoviste.json", 'w') as f:
    json.dump(tgv_json, f)

with open("camin.json", 'w') as f:
    json.dump(camin_json, f)

# Citirea din fisier

with open("targoviste.json", 'r') as g:
    tgv = json.load(g)

with open("camin.json", 'r') as g:
    camin = json.load(g)

In [95]:
# Aici sunt niste variabile definite pentru usurinta
tgv["locatie"] = 'targoviste'
camin["locatie"] = 'camin'
fields = ('ora', 'ziua', 'luna', 'an', 'index') # campurile dorite la fisierele csv

# For-ul de mai jos trebuie rulat doar o data, pentru a crea fisierele fiecarei locatii cu headerele necesare

In [96]:
# for zona in tgv, camin:
#     nume_fisier = zona["locatie"] + ".csv"
#     with open(nume_fisier, 'w') as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=fields)
#         writer.writeheader() # aici scriem doar headerele, ca apoi sa dam append la linii

Acum, vom scrie doar datele de care avem nevoie in fisiere .csv, unul per locatie.

In [97]:
# In sfarsit, aici scriem bucatile necesare in csv
for zona in tgv, camin:
    nume_fisier = zona["locatie"] + ".csv"
    for old in zona["history"]:
        index = old["indexes"][0]["value"] # valoarea indicelui CAQI
        ora = old['tillDateTime'][11:13] # ora (doua cifre)
        ora = int(ora) # converteste orele in numere, pentru orele de o singura cifra care altfel ar incepe cu 0
        ziua = old['tillDateTime'][8:10] # data (extragem doar ziua)
        ziua = int(ziua)
        luna = old['tillDateTime'][5:7]
        luna = int(luna)
        an = old['tillDateTime'][0:4]
        an = int(an)
        old_dict = {'ora': ora, 'ziua': ziua, 'luna': luna, 'an': an, 'index': index} # dictionarul de scris in fisier

        with open(nume_fisier, 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fields)
            writer.writerow(old_dict)

In [98]:
# Citim datele
date_tgv = pd.read_csv("targoviste.csv")
date_camin = pd.read_csv("camin.csv")

# Scoatem datele identice si pe cele nemasurate
def clean(date):
    date = date.drop_duplicates() # dubluri
    date = date.dropna() # valori inexistente
    return date
date_tgv = clean(date_tgv)
date_camin = clean(date_camin)

# Le copiem inapoi in stocare pentru backup, fara a adauga o coloana inutila de indecsi
date_tgv.to_csv("targoviste.csv", index=False, header=True)
date_camin.to_csv("camin.csv", index=False, header=True)

In [99]:
# Creem un dataframe combinand cele doua seturi de date, totusi asigurand ca le putem diferentias

date_adunate = pd.concat([date_tgv.assign(dataset='targoviste'), date_camin.assign(dataset='camin')])
date_adunate.loc[date_adunate['ora'] + 2 > 23, 'ziua'] += 1 # daca trecem de ora 24, e vorba de ziua urmatoare

# Airly foloseste UTC pentru ora, deci trebuie sa convertim in ora noastra
# Am facut asta dupa ce am incrementat data, pentru ca ora va trebui si ea modificata, si nu am putut sa combin functiile prea usor
def hour_from_utc(ora):
    noua_ora = int(ora) + 2
    if noua_ora > 23:
        noua_ora -= 24
    return noua_ora

date_adunate['ora'] = date_adunate['ora'].apply(hour_from_utc)
date_adunate.head(3), date_adunate.tail(3)

(   ora  ziua  luna    an  index     dataset
 0    3     5     3  2025  25.69  targoviste
 1    4     5     3  2025  19.03  targoviste
 2    5     5     3  2025  14.75  targoviste,
      ora  ziua  luna    an  index dataset
 150    9    11     3  2025  57.62   camin
 151   10    11     3  2025  58.08   camin
 152   11    11     3  2025  61.64   camin)

In [100]:
# Pentru usurinta, creem o coloana pentru zilele saptamanii

def transform_zi(row):
    year = row['an']
    month = row['luna']
    day = row['ziua']
    return date(year, month, day).weekday()

date_adunate['weekday'] = date_adunate.apply(transform_zi, axis=1)

dict_zile = {0: 'Luni', 1: 'Marti', 2: 'Miercuri', 3: 'Joi', 4: 'Vineri', 5: 'Sambata', 6: 'Duminica'}

date_adunate['weekday'] = date_adunate['weekday'].map(dict_zile)

In [101]:
# Backup
date_adunate.to_csv("date_adunate.csv", index=False, header=True)